In [23]:
import import_ipynb
from CustomDataset import ControlsDataset

import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [24]:
dataset = ControlsDataset()
dataset.convertTOClass()
dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)
print(dataset.data_frame.head(10))

   img  steeringAng
0    1            9
1    2            9
2    3            9
3    4            9
4    5            9
5    6            9
6    7            9
7    8            9
8    9            9
9   10            9


In [25]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        img_size = torch.Size([1, 3, 480, 640]) # [batch_size, channels, height, width]
        empty = torch.zeros(img_size)
        
        # Conv2d(in_channels, out_channels, kernelSize, strides)
        # stride=3 ==> moving Filter 3 pixels between the application of kernel size
        self.conv = nn.Sequential(nn.Conv2d(3, 16, 11, stride=3),
                                  nn.MaxPool2d(2),
                                  nn.ReLU(),
                                  nn.Conv2d(16, 32, 11, stride=3),
                                  nn.MaxPool2d(2),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, 7, stride=3),
                                  nn.ReLU()
                                 )
        
        units = self.conv(empty).numel()
        print("units after conv", units)
        
        self.fc = nn.Sequential(nn.Linear(units, units//2),
                                nn.ReLU(),
                                nn.Linear(units//2, units//4),
                                nn.ReLU(),
                                nn.Linear(units//4, 20)) # <-- Returning predictions over classes
        
        print("conv parameters: ", sum(p.numel() for p in self.conv.parameters()))
        print("fc parameters: ", sum(p.numel() for p in self.fc.parameters()))
    
    def forward(self, x):
        #x: batch, channel, height, width
        batch_size = x.shape[0]
        
        out = self.conv(x)
        out = out.reshape((batch_size,-1))
        out = self.fc(out)
        print(out)
        return out
net = ConvNet()

('units after conv', 512)
('conv parameters: ', 168224)
('fc parameters: ', 166804)


In [26]:
for i, batch in enumerate(dataloader):
    if i > 0:
        break
        
    imgs = batch['image'].float()
    print("input", imgs.shape)
    out = net(imgs)
    print("output", out.shape)

('input', torch.Size([256, 3, 480, 640]))


RuntimeError: size mismatch, m1: [32768 x 4], m2: [512 x 256] at ../aten/src/TH/generic/THTensorMath.cpp:136